In [1]:
import numpy as np
from sklearn import datasets
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_predict, cross_validate, cross_val_score
from sklearn.model_selection import KFold
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from pprint import pprint
from sklearn.svm import SVC
import pandas as pd

In [2]:
# Carga de datos.
datos = datasets.load_iris()
# pprint(datos)
print(np.shape(datos.data))

(150, 4)


In [3]:
# Extraer la matriz de datos "X" y la variable target "y"
X = datos.data
y = datos.target

In [4]:
# Carga de datos.
csv = pd.read_csv('Dataset_spine_2C.csv', sep=',')
print(csv.head())
# Convertir dataframe en numpy array
data = np.array(csv)
X = data[:,:-1]
y = data[:,-1]

ocur, frec = np.unique(y, return_counts=True)
print(ocur, frec)

FileNotFoundError: [Errno 2] No such file or directory: 'Dataset_spine_2C.csv'

In [5]:
# Partición EXTERNA. Test: hold-out split 80-20%.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('Train dimensions: ', np.shape(X_train))
print('Test dimensions:  ', np.shape(X_test))

Train dimensions:  (120, 4)
Test dimensions:   (30, 4)


In [ ]:
# Estandarización de los datos de entrenamiento y test
standardizer = preprocessing.StandardScaler()
X_train = standardizer.fit_transform(X_train)
X_test = standardizer.transform(X_test)

In [ ]:
# Definimos el algoritmo
# C=0.1, gamma='auto' y kernel gausiano
alg = SVC(C=1, gamma='scale', kernel='rbf')

In [ ]:
# Validación y obtención de las predicciones del modelo.
# en "cv=KFold(n_splits=5)" se hace el cross-validation INTERNO!!
# el comando "cross_val_predict" nos permite extraer directamente las predicciones de las 5 bolsas de validación
y_pred = cross_val_predict(alg, X_train, y_train, cv = KFold(n_splits=5, shuffle=True, random_state=42))
print(y_pred)

In [ ]:
# Presentación de los resultados de la evaluación.
print("Exactitud: %.3f" % (metrics.accuracy_score(y_train, y_pred))) # accuracy
print("Precisión: %.3f" % (metrics.precision_score(y_train, y_pred, average="macro"))) # precision
print("Sensibilidad: %.3f" % (metrics.recall_score(y_train, y_pred, average="macro"))) # sensibilidad
print("F1-score: %.3f" % (metrics.f1_score(y_train, y_pred, average="macro"))) # F-score


In [ ]:
# Extraemos la matriz de confusión
print("Matriz de confusión:\n", metrics.confusion_matrix(y_train, y_pred))

In [ ]:
print("Tabla de métricas:\n", metrics.classification_report(y_train, y_pred))

In [ ]:
# Una vez hemos optimizado los hiperparámetros con el conjunto de validación, utilizamos todos los datos de 
# "train" y "validation" para entrenar un modelo definitivo con esos hiperparámetros ajustados
model = SVC(C=1, gamma='scale', kernel='rbf') # Definimos el modelo con los hiperparámetros óptimos
model = model.fit(X_train, y_train) # Entrenamos el modelo con todos los datos de "train" + "val"

# y finalmente evaluamos el test con el método "score()"
test_results = model.score(X_test, y_test) # Evaluamos la exactitud del test
print('Exactitud en test: ', np.round(test_results*100,4), '%')

In [ ]:
# obtenemos las predicciones del test para extraer el resto de métricas de interés
y_pred_test = model.predict(X_test) # Extraer predicciones
print(metrics.classification_report(y_test, y_pred_test)) # Aplicar el método de classification_report()
print(metrics.confusion_matrix(y_test, y_pred_test)) # Extraer la matriz de confusión